## **Install Package**

In [ ]:
%pip install https://github.com/renan-peres/fabric-remote-tools/raw/main/fabric_remote_tools-0.1.1.tar.gz
%pip install git+https://github.com/renan-peres/fabric-remote-tools@main

## **Load Environmet Variables (.env File)**

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

account_name = os.getenv("ACCOUNT_NAME")
workspace_id = os.getenv("WORKSPACE_ID")
lakehouse_id = os.getenv("LAKEHOUSE_ID")
organization_url = os.getenv("ORGANIZATIONAL_URL")
personal_access_token = os.getenv("PERSONAL_ACCESS_TOKEN")
project_name = os.getenv("PROJECT_NAME")
repo_name = os.getenv("REPO_NAME")
github_token = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")
github_username = os.getenv("GITHUB_USERNAME")

## **Import Modules & Get Authentication Token**

In [50]:
from fabric_remote_tools import FabricAuth, OneLakeUtils

# Create an instance of FabricAuth
fabric_auth = FabricAuth()

# Create an instance of OneLakeUtils
onelake_ops = OneLakeUtils()

# Get Authentication Token
token = fabric_auth.get_service_principal_token()

# Get File System Client
file_system_client = fabric_auth.get_file_system_client(token, account_name, workspace_id)

## **Write to Lakehouse (Files/Tables)**

### **Local Tables (Delta)**

In [ ]:
# Single Table
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client
    ,upload_from="local"
    ,source_path="../assets/data/Tables/venture_funding_deals_delta"
    ,target_path="Tables/local_venture_funding_deals_delta"
)
 
# Multiple Tables in a Folder
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client
    ,upload_from="local"
    ,source_path="../assets/data/Tables"
    ,target_path="Tables/"
)

### **Local Files/Folders**

In [ ]:
# Whole Folder
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client
    ,upload_from="local"
    ,source_path="../assets/data/Files"
    ,target_path="Files/"
)

# Individual Subfolder inside a Folder
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client
    ,upload_from="local"
    ,source_path="../assets/data/Contoso"
    ,target_path="Files/Contoso"
)

# Specific File in a Folder
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client
    ,upload_from="local"
    ,source_path="../assets/data/Files/Contoso/contoso_sales.csv"
    ,target_path="Files/Contoso/contoso_sales.csv"
)

### **GitHub (Public Repo)** 

In [ ]:
# Whole GitHub repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="github",
    source_path="https://github.com/renan-peres/Polars-Cookbook.git",
    target_path="Files/GitHub/Polars-Cookbook"
)

# Single Table (Delta) in Repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="github",
    source_path="https://github.com/renan-peres/Polars-Cookbook.git",
    target_path="Tables/github_venture_funding_deals_delta",
    folder_path="data/venture_funding_deals_delta"
)

# Specific folder from GitHub repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="github",
    source_path="https://github.com/renan-peres/Polars-Cookbook.git",
    target_path="Files/GitHub/data",
    folder_path="data"
)

### **GitHub (Private Repo)** 

In [ ]:
# Whole GitHub private repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="github_private",
    repo_name=os.getenv("GITHUB_REPO_NAME"),
    target_path=f"Files/GitHub/{os.getenv('GITHUB_REPO_NAME')}"
)

# Specific folder from GitHub private repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="github_private",
    repo_name=os.getenv("GITHUB_REPO_NAME"),
    target_path="Files/GitHub/data",
    folder_path="data"
)

### **Azure DevOps (Private Repo)**

In [ ]:
# Whole Azure DevOps repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="azure_devops",
    project_name=os.getenv("PROJECT_NAME"),
    repo_name=os.getenv("REPO_NAME"),
    target_path=f"Files/AzureDevOps/{os.getenv('REPO_NAME')}"
)

# Specific folder from Azure DevOps repository
onelake_ops.write_to_lakehouse(
    file_system_client=file_system_client,
    upload_from="azure_devops",
    project_name=os.getenv("PROJECT_NAME"),
    repo_name=os.getenv("REPO_NAME"),
    target_path="Files/AzureDevOps/data",
    folder_path="/data"
)

## **List Items from Lakehouse (Files/Tables)**

In [ ]:
# List All Items in Lakehouse
onelake_ops.list_items(file_system_client=file_system_client
                     ,target_directory_path="Tables" # Tables or Files
                    #  ,print_output= True # Optional
                     )

## **Read Delta Table from Lakehouse**

In [ ]:
import polars as pl

local_path = onelake_ops.read_delta_from_fabric_lakehouse(
    file_system_client=file_system_client
    ,target_file_path="Tables/venture_funding_deals_delta_partitioned"
)

# Read Download Delta Tables
df = pl.scan_delta(local_path).collect()

# Delete the downloaded files
onelake_ops.delete_local_path(local_path)

# Display DataFrame
display(df)

## **Download Items from Lakehouse (Files/Tables)**

In [ ]:
# Tables
onelake_ops.download_from_lakehouse(
    file_system_client=file_system_client
    # ,target_file_path="Tables/venture_funding_deals" # Single Table
    ,target_file_path="Tables/" # All Tables
)

# Files
onelake_ops.download_from_lakehouse(
    file_system_client=file_system_client
    # ,target_file_path="Files/Contoso/contoso_sales.csv" # Single File
    # ,target_file_path="Files/Contoso/" # Subfolder
    ,target_file_path="Files/" # All Subfolders & Files
)

## **Delete Items from Lakehouse (Files/Tables)**

In [ ]:
# Tables
onelake_ops.delete_file(
    file_system_client=file_system_client
    # ,lakehouse_dir_path="Tables/venture_funding_deals_delta" # Single Table
    ,lakehouse_dir_path="Tables/" # All Tables
)

# Files
onelake_ops.delete_file(
    file_system_client=file_system_client 
    # ,lakehouse_dir_path="Files/Contoso/contoso_sales.csv" # Single File
    # ,lakehouse_dir_path="Files/Contoso" # Subfolder
    ,lakehouse_dir_path="Files/" # All Subfolders & Files
)